In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Téléchargement des ressources NLTK pour analyser les sentiments
nltk.download('vader_lexicon')

# Initialiser l'analyseur de sentiments de NLTK
sia = SentimentIntensityAnalyzer()

# Charger un modèle pré-entraîné GPT-2 pour la génération de texte
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Pipeline de détection des commentaires haineux ou discriminatoires
classifier = pipeline('text-classification', model='unitary/toxic-bert')

# Fonction pour détecter si le commentaire est haineux et sa catégorie
def detect_toxicity(comment):
    result = classifier(comment)[0]
    if result['label'] == 'TOXICITY' and result['score'] > 0.5:
        return True, result['label']
    return False, None

# Fonction pour générer un texte reformulé
def generate_positive_reformulation(comment, toxicity_label=None):
    # Vérification du sentiment du texte avec NLTK
    sentiment_score = sia.polarity_scores(comment)

    # Reformuler si c'est malveillant ou si c'est un discours haineux détecté par le modèle
    if sentiment_score['compound'] < -0.5 or toxicity_label:
        # Ajouter un prompt en fonction de la catégorie du discours haineux
        if toxicity_label == 'TOXICITY':
            prompt = "Reformule ce commentaire haineux en un message respectueux : " + comment
        else:
            prompt = "Reformule ce commentaire de manière positive : " + comment

        # Encoder l'entrée pour le modèle
        inputs = tokenizer.encode(prompt, return_tensors="pt")

        # Générer la réponse
        outputs = model.generate(
            inputs,
            max_length=150,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_p=0.95,
            temperature=0.75,
        )

        # Décoder la sortie générée par le modèle
        reformulated_comment = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Filtrer les éventuelles expressions négatives dans la reformulation générée
        if 'stupid' in reformulated_comment or 'inutile' in reformulated_comment:
            reformulated_comment = reformulated_comment.replace('stupid', 'non pertinent')
            reformulated_comment = reformulated_comment.replace('inutile', 'moins utile')

        return reformulated_comment
    else:
        return comment

# Exemple de commentaires malveillants
comments = [
    "Cette personne est vraiment stupide et inutile.",
    "Je déteste absolument ce groupe, ils n'apportent rien de bon.",
    "Tu es tellement incompétent, tu ne devrais pas exister.",
    "Cette race est inférieure, ils ne devraient pas être ici."
]

# Reformuler les commentaires
for comment in comments:
    is_toxic, toxicity_label = detect_toxicity(comment)
    positive_comment = generate_positive_reformulation(comment, toxicity_label)
    print(f"Commentaire original : {comment}")
    print(f"Reformulation positive : {positive_comment}")
    print("-" * 50)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Commentaire original : Cette personne est vraiment stupide et inutile.
Reformulation positive : Cette personne est vraiment stupide et inutile.
--------------------------------------------------
Commentaire original : Je déteste absolument ce groupe, ils n'apportent rien de bon.
Reformulation positive : Je déteste absolument ce groupe, ils n'apportent rien de bon.
--------------------------------------------------
Commentaire original : Tu es tellement incompétent, tu ne devrais pas exister.
Reformulation positive : Tu es tellement incompétent, tu ne devrais pas exister.
--------------------------------------------------
Commentaire original : Cette race est inférieure, ils ne devraient pas être ici.
Reformulation positive : Cette race est inférieure, ils ne devraient pas être ici.
--------------------------------------------------


In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Téléchargement des ressources NLTK pour analyser les sentiments
nltk.download('vader_lexicon')

# Initialiser l'analyseur de sentiments de NLTK
sia = SentimentIntensityAnalyzer()

# Charger un modèle pré-entraîné GPT-2 pour la génération de texte
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Fonction pour générer un texte reformulé
def generate_positive_reformulation(comment):
    # Vérification du sentiment du texte
    sentiment_score = sia.polarity_scores(comment)

    # Si le sentiment est négatif ou haineux, on le reformule
    if sentiment_score['compound'] < -0.5:
        # Ajouter un prompt pour guider le modèle vers une reformulation positive
        prompt = "Reformule ce commentaire de manière positive : " + comment

        # Encoder l'entrée pour le modèle
        inputs = tokenizer.encode(prompt, return_tensors="pt")

        # Générer la réponse
        outputs = model.generate(
            inputs,
            max_length=150,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_p=0.95,
            temperature=0.75,
        )

        # Décoder la sortie générée par le modèle
        reformulated_comment = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return reformulated_comment
    else:
        return comment

# Exemple de commentaires malveillants
comments = [
    "Cette personne est vraiment stupide et inutile.",
    "Je déteste absolument ce groupe, ils n'apportent rien de bon.",
    "Tu es tellement incompétent, tu ne devrais pas exister."
]

# Reformuler les commentaires
for comment in comments:
    positive_comment = generate_positive_reformulation(comment)
    print(f"Commentaire original : {comment}")
    print(f"Reformulation positive : {positive_comment}")
    print("-" * 50)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Commentaire original : Cette personne est vraiment stupide et inutile.
Reformulation positive : Cette personne est vraiment stupide et inutile.
--------------------------------------------------
Commentaire original : Je déteste absolument ce groupe, ils n'apportent rien de bon.
Reformulation positive : Je déteste absolument ce groupe, ils n'apportent rien de bon.
--------------------------------------------------
Commentaire original : Tu es tellement incompétent, tu ne devrais pas exister.
Reformulation positive : Tu es tellement incompétent, tu ne devrais pas exister.
--------------------------------------------------


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Téléchargement des ressources NLTK pour analyser les sentiments
nltk.download('vader_lexicon')

# Initialiser l'analyseur de sentiments de NLTK
sia = SentimentIntensityAnalyzer()

# Charger un modèle pré-entraîné GPT-2 pour la génération de texte
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Fonction pour vérifier si une sortie contient des mots négatifs
def contains_negative_words(comment):
    negative_words = ['stupide', 'inutile', 'nul', 'incompétent', 'haine', 'idiot', 'déteste']
    return any(word in comment.lower() for word in negative_words)

# Fonction pour générer un texte reformulé
def generate_positive_reformulation(comment):
    # Vérification du sentiment du texte
    sentiment_score = sia.polarity_scores(comment)

    # Si le sentiment est négatif ou haineux, on le reformule
    if sentiment_score['compound'] < -0.5:
        # Ajouter un prompt pour guider le modèle vers une reformulation positive
        prompt = "Transforme ce commentaire malveillant en un message respectueux et encourageant : " + comment

        # Encoder l'entrée pour le modèle
        inputs = tokenizer.encode(prompt, return_tensors="pt")

        # Générer la réponse
        outputs = model.generate(
            inputs,
            max_length=150,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_p=0.9,  # Ajustement du nucleus sampling
            temperature=0.7,  # Ajustement de la créativité
        ),

        # Décoder la sortie générée par le modèle
        reformulated_comment = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Vérification de la présence de mots négatifs dans la sortie
        if contains_negative_words(reformulated_comment):
            reformulated_comment = "La reformul ation n'a pas abouti à un commentaire respectueux, veuillez réessayer."

        return reformulated_comment
    else:
        return comment

# Exemple de commentaires malveillants
comments = [
    "Cette personne est vraiment stupide et inutile.",
    "Je déteste absolument ce groupe, ils n'apportent rien de bon.",
    "Tu es tellement incompétent, tu ne devrais pas exister."
]

# Reformuler les commentaires
for comment in comments:
    positive_comment = generate_positive_reformulation(comment)
    print(f"Commentaire original : {comment}")
    print(f"Reformulation positive : {positive_comment}")
    print("-" * 50)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Commentaire original : Cette personne est vraiment stupide et inutile.
Reformulation positive : Cette personne est vraiment stupide et inutile.
--------------------------------------------------
Commentaire original : Je déteste absolument ce groupe, ils n'apportent rien de bon.
Reformulation positive : Je déteste absolument ce groupe, ils n'apportent rien de bon.
--------------------------------------------------
Commentaire original : Tu es tellement incompétent, tu ne devrais pas exister.
Reformulation positive : Tu es tellement incompétent, tu ne devrais pas exister.
--------------------------------------------------


In [1]:
from transformers import pipeline

# Chargement du pipeline de classification
classifier = pipeline('text-classification', model='nlptown/bert-base-multilingual-uncased-sentiment')

# Exemple de commentaire
commentaire = "Ce produit est horrible, il est totalement inutilisable."

# Classifier le commentaire
resultat = classifier(commentaire)[0]
label = resultat['label']

# Vérifier si le commentaire est malveillant
# Ici, nous supposons que si la note est inférieure à 3, le commentaire est malveillant
score = int(label.split()[0])  # Extraire la note
if score <= 2:
    print("Commentaire malveillant détecté")
else:
    print("Commentaire non malveillant")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Commentaire malveillant détecté


In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Chargement du modèle T5 pour la reformulation
model_name = "t5-small"  # Utiliser un modèle plus grand pour des résultats plus qualitatifs
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def reformuler_commentaire(commentaire):
    # Préparer l'entrée pour la reformulation
    entree = f"rephrase: {commentaire}"
    inputs = tokenizer.encode(entree, return_tensors="pt", max_length=512, truncation=True)

    # Générer la reformulation
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)

    # Décoder le résultat
    commentaire_reformule = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return commentaire_reformule

# Exemple d'utilisation
commentaire_negatif = "Ce produit est horrible, il est totalement inutilisable."
commentaire_reformule = reformuler_commentaire(commentaire_negatif)
print("Commentaire reformulé :", commentaire_reformule)
def transformer_commentaire(commentaire):
    # Étape 1 : Classifier le commentaire
    resultat = classifier(commentaire)[0]
    score = int(resultat['label'].split()[0])

    if score <= 2:  # Si le commentaire est malveillant
        print("Malveillant détecté : Reformulation...")
        commentaire_reformule = reformuler_commentaire(commentaire)
        print("Commentaire reformulé :", commentaire_reformule)
    else:
        print("Commentaire non malveillant :", commentaire)

# Exemple d'utilisation
commentaire = "Ce produit est horrible, il est totalement inutilisable."
transformer_commentaire(commentaire)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Commentaire reformulé : rephrase: Ce produit est horrible, il est totalement inutilisable.
Malveillant détecté : Reformulation...
Commentaire reformulé : rephrase: Ce produit est horrible, il est totalement inutilisable.


In [5]:
from transformers import pipeline

# Utiliser GPT-2 au lieu de T5 pour la génération de texte plus créatif
generator = pipeline('text-generation', model='gpt2')

# Fonction pour reformuler un commentaire avec humour
def reformulate_with_humor(comment):
    # Création du prompt pour guider le modèle
    prompt = f"Transform this negative comment into something funny: {comment}"

    # Générer la version humoristique
    result = generator(prompt, max_length=50, num_return_sequences=1, pad_token_id=50256)

    # Retourner le texte généré
    return result[0]['generated_text']

# Liste de commentaires négatifs
comments = [
    "Ce film est nul, le scénario est tellement prévisible.",
    "Le service client est horrible, je n'ai jamais été aussi mal traité.",
    "Cette application est une perte de temps, rien ne fonctionne."
]

# Reformuler chaque commentaire avec une version humoristique
for comment in comments:
    humoristic_comment = reformulate_with_humor(comment)
    print(f"Commentaire original: {comment}")
    print(f"Version humoristique : {humoristic_comment}\n")


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Commentaire original: Ce film est nul, le scénario est tellement prévisible.
Version humoristique : Transform this negative comment into something funny: Ce film est nul, le scénario est tellement prévisible.

It's not much of a problem on its own, but not for people who are not big fans of movies that

Commentaire original: Le service client est horrible, je n'ai jamais été aussi mal traité.
Version humoristique : Transform this negative comment into something funny: Le service client est horrible, je n'ai jamais été aussi mal traité. Je ne retour dans le petit mal: c'est une réseau,

Commentaire original: Cette application est une perte de temps, rien ne fonctionne.
Version humoristique : Transform this negative comment into something funny: Cette application est une perte de temps, rien ne fonctionne.

"That's right! If someone wants to go to you or any way they tell you to go out



In [6]:
from transformers import pipeline

# Utiliser GPT-2 pour la génération de texte
generator = pipeline('text-generation', model='gpt2')

# Fonction pour reformuler un commentaire avec humour
def reformulate_with_humor(comment):
    # Création d'un prompt plus spécifique pour guider le modèle
    prompt = f"Take the following negative comment and make it funny and sarcastic: '{comment}'"

    # Générer la version humoristique avec troncature et une longueur limitée
    result = generator(prompt, max_length=60, num_return_sequences=1, pad_token_id=50256, truncation=True)

    # Retourner le texte généré
    return result[0]['generated_text'].replace(prompt, '').strip()

# Liste de commentaires négatifs
comments = [
    "Ce film est nul, le scénario est tellement prévisible.",
    "Le service client est horrible, je n'ai jamais été aussi mal traité.",
    "Cette application est une perte de temps, rien ne fonctionne."
]

# Reformuler chaque commentaire avec une version humoristique
for comment in comments:
    humoristic_comment = reformulate_with_humor(comment)
    print(f"Commentaire original: {comment}")
    print(f"Version humoristique : {humoristic_comment}\n")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Commentaire original: Ce film est nul, le scénario est tellement prévisible.
Version humoristique : The author of Le Scénario Quotes has recently added a line to his novel 'Egalité', which seems slightly more appropriate

Commentaire original: Le service client est horrible, je n'ai jamais été aussi mal traité.
Version humoristique : I'd rather not ask. My main job as a journalist — and the one I want to make sure the

Commentaire original: Cette application est une perte de temps, rien ne fonctionne.
Version humoristique : And read the following comment and make it more like "You should never take a joke seriously — or even consider it — but use



In [7]:
from transformers import pipeline

# Utiliser GPT-2 pour la génération de texte (il existe des versions multilingues)
generator = pipeline('text-generation', model='gpt2')

# Fonction pour reformuler un commentaire avec humour en français
def reformulate_with_humor(comment):
    # Création d'un prompt plus spécifique et en français
    prompt = f"Prends ce commentaire négatif et rends-le drôle et sarcastique : '{comment}'"

    # Générer la version humoristique avec troncature et une longueur limitée
    result = generator(prompt, max_length=60, num_return_sequences=1, pad_token_id=50256, truncation=True)

    # Retourner le texte généré en supprimant le prompt de départ
    return result[0]['generated_text'].replace(prompt, '').strip()

# Liste de commentaires négatifs en français
comments = [
    "Ce film est nul, le scénario est tellement prévisible.",
    "Le service client est horrible, je n'ai jamais été aussi mal traité.",
    "Cette application est une perte de temps, rien ne fonctionne."
]

# Reformuler chaque commentaire avec une version humoristique
for comment in comments:
    humoristic_comment = reformulate_with_humor(comment)
    print(f"Commentaire original: {comment}")
    print(f"Version humoristique : {humoristic_comment}\n")


Commentaire original: Ce film est nul, le scénario est tellement prévisible.
Version humoristique : 3 La pête de sa pête de côté étr

Commentaire original: Le service client est horrible, je n'ai jamais été aussi mal traité.
Version humoristique : « Je n'a pas le monde qui se sait dé

Commentaire original: Cette application est une perte de temps, rien ne fonctionne.
Version humoristique : Père l'obscenêtre : ce temps si une pl



In [8]:
from transformers import pipeline

# Charger le modèle GPT-2 pré-entraîné en français
generator = pipeline('text-generation', model='dbddv01/gpt2-french-small')

# Fonction pour reformuler un commentaire avec humour en français
def reformulate_with_humor(comment):
    # Création d'un prompt plus spécifique en français
    prompt = f"Transforme ce commentaire négatif en quelque chose de drôle et sarcastique : '{comment}'"

    # Générer la version humoristique avec troncature et longueur limitée
    result = generator(prompt, max_length=60, num_return_sequences=1, pad_token_id=50256, truncation=True)

    # Retourner le texte généré en supprimant le prompt de départ
    return result[0]['generated_text'].replace(prompt, '').strip()

# Liste de commentaires négatifs en français
comments = [
    "Ce film est nul, le scénario est tellement prévisible.",
    "Le service client est horrible, je n'ai jamais été aussi mal traité.",
    "Cette application est une perte de temps, rien ne fonctionne."
]

# Reformuler chaque commentaire avec une version humoristique
for comment in comments:
    humoristic_comment = reformulate_with_humor(comment)
    print(f"Commentaire original: {comment}")
    print(f"Version humoristique : {humoristic_comment}\n")


config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/858k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/517k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Commentaire original: Ce film est nul, le scénario est tellement prévisible.
Version humoristique : Les premières saisons, avec trois épisodes à l'avance, montrent le talent, mais une situation assez grave qui peut faire perdre le rythme tout en restant encore un

Commentaire original: Le service client est horrible, je n'ai jamais été aussi mal traité.
Version humoristique : »La race est désormais présente en Allemagne comme très fortement au sud de la Norvège. Dans le sud-ouest du pays, la population est

Commentaire original: Cette application est une perte de temps, rien ne fonctionne.
Version humoristique : Il est donc difficile de comprendre comment un système (bée) n'est pas possible par rapport au système (a) ou à l'autre. »



In [10]:
from transformers import pipeline

# Pipeline pour la détection des insultes et commentaires malveillants
insult_detector = pipeline('text-classification', model="mrm8488/bert-mini-finetuned-age_news-classification")

# Pipeline de génération humoristique avec GPT-2 en français
humor_generator = pipeline('text-generation', model='dbddv01/gpt2-french-small')

# Fonction pour détecter si un commentaire est insultant ou négatif
def is_insulting_or_negative(comment):
    # Utiliser le modèle de classification pour détecter les insultes
    result = insult_detector(comment)

    # Examiner les résultats (on suppose que le label 'insult' ou 'toxic' est détecté)
    for res in result:
        if res['label'] in ['toxic', 'insult', 'negative'] and res['score'] > 0.5:
            return True
    return False

# Fonction pour reformuler un commentaire avec humour
def reformulate_with_humor(comment):
    # Création d'un prompt plus spécifique en français
    prompt = f"Transforme ce commentaire négatif en quelque chose de drôle et sarcastique : '{comment}'"

    # Générer la version humoristique
    result = humor_generator(prompt, max_length=60, num_return_sequences=1, pad_token_id=50256, truncation=True)

    # Retourner le texte généré en supprimant le prompt de départ
    return result[0]['generated_text'].replace(prompt, '').strip()

# Liste de commentaires à tester
comments = [
    "sale pute.",
    "Le service client est horrible, je n'ai jamais été aussi mal traité.",
    "Cette application est une perte de temps, rien ne fonctionne.",
    "Tu es vraiment stupide et inutile.",
    "Ce restaurant est une honte, tout était dégoûtant."
]

# Processus principal : détection d'insultes puis reformulation humoristique
for comment in comments:
    if is_insulting_or_negative(comment):
        humoristic_comment = reformulate_with_humor(comment)
        print(f"Commentaire original: {comment}")
        print(f"Version humoristique : {humoristic_comment}\n")
    else:
        print(f"Commentaire original: {comment}")
        print("Aucun contenu malveillant détecté, pas de reformulation nécessaire.\n")


Commentaire original: sale pute.
Aucun contenu malveillant détecté, pas de reformulation nécessaire.

Commentaire original: Le service client est horrible, je n'ai jamais été aussi mal traité.
Aucun contenu malveillant détecté, pas de reformulation nécessaire.

Commentaire original: Cette application est une perte de temps, rien ne fonctionne.
Aucun contenu malveillant détecté, pas de reformulation nécessaire.

Commentaire original: Tu es vraiment stupide et inutile.
Aucun contenu malveillant détecté, pas de reformulation nécessaire.

Commentaire original: Ce restaurant est une honte, tout était dégoûtant.
Aucun contenu malveillant détecté, pas de reformulation nécessaire.



In [20]:
from transformers import pipeline

# Charger un modèle de classification pour détecter les insultes et le discours haineux
# Utiliser un modèle spécifique pour la détection de la toxicité
insult_detector = pipeline('text-classification', model='unitary/toxic-bert')

# Fonction pour détecter les insultes graves et le discours discriminatoire
def is_toxic(comment):
    result = insult_detector(comment)

    # Vérifier les résultats pour voir s'il y a des insultes ou des discours de haine
    for res in result:
        if res['label'] == 'TOXIC' and res['score'] > 0.7:  # seuil de confiance de 70%
            return True, res['score']  # Retourne True et le score de toxicité
        elif res['label'] == 'SEVERE_TOXIC' and res['score'] > 0.7:  # seuil pour les insultes graves
            return True, res['score']  # Retourne True et le score de toxicité
    return False, 0  # Aucun commentaire malveillant détecté

# Liste de commentaires à tester
comments = [
    "Tu es vraiment stupide et inutile.",
    "Ce film est nul, le scénario est tellement prévisible.",
    "Je déteste les gens de cette communauté.",
    "Le service client est horrible, je n'ai jamais été aussi mal traité.",
    "Cette application est une perte de temps, rien ne fonctionne.",
    "Les femmes sont toutes des manipulatrices.",
    "Ce restaurant est une honte, tout était dégoûtant."
    "vous êtes adorable "
    "Ce film est une perte de temps, le pire que j’aie jamais vu.",
    "Les gens de cette communauté ne sont que des hypocrites.",
    "Ton avis est complètement débile.",
    "Cette application est nulle, elle ne fonctionne jamais.",
    "Pourquoi es-tu si mauvais dans ce que tu fais ?"
    "Ce restaurant est dégoûtant, je ne peux pas croire qu'il existe encore.",
    "Les femmes ne savent rien faire correctement.",
    "Franchement, qui a eu l'idée de produire ce navet ?",
    "Tu es une perte de temps et d'espace.",
    "Je pense que tu peux mieux faire, continue à essayer !",
    "Ce film a ses défauts, mais il a aussi des moments intéressants.",
    "Chaque communauté a ses défis, mais j'aime la diversité des opinions.",
    "Ton point de vue est différent, mais il est intéressant d'entendre d'autres perspectives.",
    "Cette application a du potentiel, j'espère qu'elle s'améliorera avec le temps.",
    "Nous avons tous nos forces et faiblesses, et c'est ce qui fait de nous des humains.",
    "Je suis sûr que ce restaurant peut améliorer ses plats pour plaire à tous.",
    "Il est important de juger chaque personne sur ses actions et non sur son genre.",
    "Ce film pourrait toucher des cordes sensibles pour certains, ce qui est positif.",
    "Je suis sûr que tu vas progresser avec un peu de pratique et de patience."
        ]

# Processus principal : détection d'insultes et de propos discriminatoires
for comment in comments:
    is_insulting, score = is_toxic(comment)
    if is_insulting:
        print(f"Commentaire détecté comme acceptable  : '{comment}' avec un score de toxicité de {score:.2f}")
    else:
        print(f"Commentaire détecté : '{comment}'")


Commentaire détecté : 'Tu es vraiment stupide et inutile.'
Commentaire détecté : 'Ce film est nul, le scénario est tellement prévisible.'
Commentaire détecté : 'Je déteste les gens de cette communauté.'
Commentaire détecté : 'Le service client est horrible, je n'ai jamais été aussi mal traité.'
Commentaire détecté : 'Cette application est une perte de temps, rien ne fonctionne.'
Commentaire détecté : 'Les femmes sont toutes des manipulatrices.'
Commentaire détecté : 'Ce restaurant est une honte, tout était dégoûtant.vous êtes adorable Ce film est une perte de temps, le pire que j’aie jamais vu.'
Commentaire détecté : 'Les gens de cette communauté ne sont que des hypocrites.'
Commentaire détecté : 'Ton avis est complètement débile.'
Commentaire détecté : 'Cette application est nulle, elle ne fonctionne jamais.'
Commentaire détecté : 'Pourquoi es-tu si mauvais dans ce que tu fais ?Ce restaurant est dégoûtant, je ne peux pas croire qu'il existe encore.'
Commentaire détecté : 'Les femmes n

In [22]:
!pip install transformers torch

from transformers import pipeline

# Charger le modèle de classification pour détecter les commentaires toxiques
toxic_detector = pipeline('text-classification', model='unitary/toxic-bert')

# Fonction pour détecter si un commentaire est toxique ou bienveillant
def detect_toxicity(comment):
    result = toxic_detector(comment)

    # Vérifier les résultats pour voir s'il y a des commentaires toxiques
    for res in result:
        if res['label'] == 'TOXIC' and res['score'] > 0.7:  # seuil de confiance de 70%
            return "Toxique", res['score']  # Retourne "Toxique" et le score de toxicité
        elif res['label'] == 'NON_TOXIC':
            return "Bienveillant", res['score']  # Retourne "Bienveillant" et le score de toxicité
    return "Inconnu", 0  # Aucun jugement clair

# Liste de commentaires à tester
comments = [
    "Tu es vraiment idiot, tu ne comprends rien !",   # Toxique
    "Ce film a ses défauts, mais il a aussi des moments intéressants.",  # Bienveillant
    "Les gens de cette communauté ne sont que des hypocrites.",  # Toxique
    "Ton point de vue est différent, mais il est intéressant d'entendre d'autres perspectives.",  # Bienveillant
    "Cette application est nulle, elle ne fonctionne jamais.",  # Toxique
    "Nous avons tous nos forces et faiblesses, et c'est ce qui fait de nous des humains.",  # Bienveillant
    "Pourquoi es-tu si mauvais dans ce que tu fais ?",  # Toxique
    "Je suis sûr que tu vas progresser avec un peu de pratique et de patience.",  # Bienveillant
]

# Processus principal : détection de la toxicité des commentaires
for comment in comments:
    toxicity, score = detect_toxicity(comment)
    print(f"Commentaire : '{comment}' => Classification : {toxicity} avec un score de {score:.2f}")


Commentaire : 'Tu es vraiment idiot, tu ne comprends rien !' => Classification : Inconnu avec un score de 0.00
Commentaire : 'Ce film a ses défauts, mais il a aussi des moments intéressants.' => Classification : Inconnu avec un score de 0.00
Commentaire : 'Les gens de cette communauté ne sont que des hypocrites.' => Classification : Inconnu avec un score de 0.00
Commentaire : 'Ton point de vue est différent, mais il est intéressant d'entendre d'autres perspectives.' => Classification : Inconnu avec un score de 0.00
Commentaire : 'Cette application est nulle, elle ne fonctionne jamais.' => Classification : Inconnu avec un score de 0.00
Commentaire : 'Nous avons tous nos forces et faiblesses, et c'est ce qui fait de nous des humains.' => Classification : Inconnu avec un score de 0.00
Commentaire : 'Pourquoi es-tu si mauvais dans ce que tu fais ?' => Classification : Inconnu avec un score de 0.00
Commentaire : 'Je suis sûr que tu vas progresser avec un peu de pratique et de patience.' => 

In [14]:
from transformers import pipeline

# Charger un modèle de classification pour détecter les insultes et le discours haineux
insult_detector = pipeline('text-classification', model='unitary/toxic-bert')

# Charger le modèle de génération pour reformuler en positif
positive_generator = pipeline('text-generation', model='dbddv01/gpt2-french-small')

# Fonction pour détecter les insultes graves et le discours discriminatoire
def is_toxic(comment):
    result = insult_detector(comment)

    # Vérifier les résultats pour voir s'il y a des insultes ou des discours de haine
    for res in result:
        if res['label'] == 'TOXIC' and res['score'] > 0.7:  # seuil de confiance de 70%
            return True, res['score']  # Retourne True et le score de toxicité
        elif res['label'] == 'SEVERE_TOXIC' and res['score'] > 0.7:  # seuil pour les insultes graves
            return True, res['score']  # Retourne True et le score de toxicité
    return False, 0  # Aucun commentaire malveillant détecté

# Fonction pour reformuler un commentaire en phrase positive
def reformulate_to_positive(comment):
    # Création d'un prompt pour générer une version positive
    prompt = f"Transforme cette phrase négative en une phrase positive : '{comment}'"

    # Générer la version positive
    result = positive_generator(prompt, max_length=60, num_return_sequences=1, pad_token_id=50256, truncation=True)

    # Retourner le texte généré en supprimant le prompt de départ
    return result[0]['generated_text'].replace(prompt, '').strip()

# Liste de commentaires à tester
comments = [
    "Tu es vraiment stupide et inutile.",
    "Ce film est nul, le scénario est tellement prévisible.",
    "Je déteste les gens de cette communauté.",
    "Le service client est horrible, je n'ai jamais été aussi mal traité.",
    "Cette application est une perte de temps, rien ne fonctionne.",
    "Les femmes sont toutes des manipulatrices.",
    "Ce restaurant est une honte, tout était dégoûtant."
]

# Processus principal : détection d'insultes puis reformulation positive
for comment in comments:
    is_insulting, score = is_toxic(comment)
    if is_insulting:
        print(f"Commentaire détecté comme inacceptable : '{comment}' avec un score de toxicité de {score:.2f}")
        positive_comment = reformulate_to_positive(comment)
        print(f"Version positive : {positive_comment}\n")
    else:
        print(f"Commentaire acceptable : '{comment}'\n")


Commentaire acceptable : 'Tu es vraiment stupide et inutile.'

Commentaire acceptable : 'Ce film est nul, le scénario est tellement prévisible.'

Commentaire acceptable : 'Je déteste les gens de cette communauté.'

Commentaire acceptable : 'Le service client est horrible, je n'ai jamais été aussi mal traité.'

Commentaire acceptable : 'Cette application est une perte de temps, rien ne fonctionne.'

Commentaire acceptable : 'Les femmes sont toutes des manipulatrices.'

Commentaire acceptable : 'Ce restaurant est une honte, tout était dégoûtant.'



In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Charger le modèle T5
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def reformuler_commentaire(commentaire):
    # Préparer l'entrée avec une instruction plus spécifique
    entree = f"translate negative to positive: {commentaire}"
    inputs = tokenizer.encode(entree, return_tensors="pt", max_length=512, truncation=True)

    # Générer la reformulation
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)

    # Décoder le résultat
    commentaire_reformule = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return commentaire_reformule

# Exemple d'utilisation
commentaire_negatif = "Ce produit est horrible, il est totalement inutilisable."
commentaire_reformule = reformuler_commentaire(commentaire_negatif)
print("Commentaire reformulé :", commentaire_reformule)



Commentaire reformulé : Ce produit est horrible, il est totalement inutilisable.
